<a href="https://colab.research.google.com/github/SuehyunBaig/WiCDS-ML-Committee/blob/main/Project_Fall_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers==4.33.1
!pip install datasets torch

In [ ]:
import transformers
from transformers import pipeline
import pandas as pd
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Yelp Restaurant Reviews - 복사본.csv to Yelp Restaurant Reviews - 복사본.csv


In [ ]:
df = pd.read_csv(list(uploaded.keys())[0])
df.head()

,Yelp URL,Rating,Date,Review Text
0,https://www.yelp.com/biz/sidney-dairy-barn-sidney,5,1/22/2022,All I can say is they have very good ice cream...
1,https://www.yelp.com/biz/sidney-dairy-barn-sidney,4,6/26/2022,Nice little local place for ice cream.My favor...
2,https://www.yelp.com/biz/sidney-dairy-barn-sidney,5,8/7/2021,A delicious treat on a hot day! Staff was very...
3,https://www.yelp.com/biz/sidney-dairy-barn-sidney,4,7/28/2016,This was great service and a fun crew! I got t...
4,https://www.yelp.com/biz/sidney-dairy-barn-sidney,5,6/23/2015,This is one of my favorite places to get ice c...


In [ ]:
# Create binary lables (1 = poisitive, 0 = negative) for training
df['Label'] = df['Rating'].apply(lambda x: 1 if x >= 4 else 0)

# Stratified train/test split to keep the label distribution same as origianl dataset
train_df, test_df = train_test_split(
    df,
    test_size=0.3,
    stratify=df["Label"],
    random_state=42
)

# Convert to Hugging Face dataset
train_df = Dataset.from_pandas(train_df)
test_df = Dataset.from_pandas(test_df)
dataset = DatasetDict({
    "train": train_df,
    "test": test_df
})

dataset

DatasetDict({
    train: Dataset({
        features: ['Yelp URL', 'Rating', 'Date', 'Review Text', 'Label', '__index_level_0__'],
        num_rows: 700
    })
    test: Dataset({
        features: ['Yelp URL', 'Rating', 'Date', 'Review Text', 'Label', '__index_level_0__'],
        num_rows: 300
    })
})

In [ ]:
# Load tokenizer from a pretrained model
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Tokenize the text
def tokenize(batch):
    return tokenizer(batch["Review Text"], padding="max_length", truncation=True)

tokenized_dataset = dataset.map(tokenize, batched=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/700 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir=r"C:\Users\natal\OneDrive\문서\Kaggle",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    logging_dir="logs",
    report_to=[]
)

In [ ]:
tokenized_dataset = tokenized_dataset.rename_column("Label", "labels")
tokenized_dataset.set_format("torch")

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"]
)

In [ ]:
trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss


TrainOutput(global_step=88, training_loss=0.4614380923184482, metrics={'train_runtime': 4840.3883, 'train_samples_per_second': 0.289, 'train_steps_per_second': 0.018, 'total_flos': 185454358118400.0, 'train_loss': 0.4614380923184482, 'epoch': 2.0})

In [ ]:
# Save the trained model and tokenizer
trainer.save_model("./my_trained_model")
tokenizer.save_pretrained("./my_trained_model")

('./my_trained_model/tokenizer_config.json',
 './my_trained_model/special_tokens_map.json',
 './my_trained_model/vocab.txt',
 './my_trained_model/added_tokens.json',
 './my_trained_model/tokenizer.json')

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained("./my_trained_model")
tokenizer = AutoTokenizer.from_pretrained("./my_trained_model")

In [ ]:
predictions = trainer.predict(tokenized_dataset["test"]).predictions

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


In [ ]:
# import numpy as np
# np.save("test_predictions.npy", predictions)

In [ ]:
# predictions = np.load("test_predictions.npy")

In [ ]:
pred_labels = predictions.argmax(axis=-1)
true_labels = tokenized_dataset["test"]["labels"]

from sklearn.metrics import accuracy_score

acc = accuracy_score(true_labels, pred_labels)
print("Test Accuracy:", acc)

Test Accuracy: 0.47333333333333333


In [ ]:
import numpy as np
np.unique(true_labels, return_counts=True)

(array([0, 1]), array([144, 156]))